Simple beat crash


In [1]:
from pydub import AudioSegment
import os

def make_mixtape(folder, output="mixtape.mp3", crossfade_ms=5000):
    files = [f for f in os.listdir(folder) if f.endswith((".mp3", ".wav"))]
    files.sort()  # optional, or random.shuffle(files)
    
    mixtape = None
    for i, file in enumerate(files):
        song = AudioSegment.from_file(os.path.join(folder, file))
        song = song.set_channels(2).set_frame_rate(44100)  # standardize
        song = song.apply_gain(-3)  # reduce loudness slightly
        
        if mixtape is None:
            mixtape = song
        else:
            mixtape = mixtape.append(song, crossfade=crossfade_ms)
    
    mixtape.export(output, format="mp3")
    print(f"✅ Mixtape generated: {output}")

# Example usage
make_mixtape("mixtape")



✅ Mixtape generated: mixtape.mp3


advanced test 

In [2]:
from pydub import AudioSegment
import os, random

def smooth_mixtape(folder, output="mixtape.mp3", crossfade_ms=10000):
    files = [f for f in os.listdir(folder) if f.endswith((".mp3", ".wav"))]
    random.shuffle(files)  # shuffle playlist
    
    mixtape = None
    for i, file in enumerate(files):
        song = AudioSegment.from_file(os.path.join(folder, file))
        song = song.set_channels(2).set_frame_rate(44100)

        if mixtape is None:
            mixtape = song
        else:
            # create transition part
            overlap = min(crossfade_ms, len(song), len(mixtape))
            
            # apply filters gradually (simulate DJ cutoff effect)
            outgoing = mixtape[-overlap:].high_pass_filter(200)  # cut bass on old
            incoming = song[:overlap].low_pass_filter(4000)      # cut highs on new

            transition = outgoing.overlay(incoming)
            
            mixtape = mixtape[:-overlap] + transition + song[overlap:]
    
    mixtape.export(output, format="mp3")
    print(f"✅ Smooth mixtape generated: {output}")

# Example usage
smooth_mixtape("mixtape")


✅ Smooth mixtape generated: mixtape.mp3


pt 2


In [4]:
from pydub import AudioSegment
import os, random

def smooth_fade_mixtape(folder, output="mixtape2.mp3", transition_ms=6000):
    files = [f for f in os.listdir(folder) if f.endswith((".mp3", ".wav"))]
    random.shuffle(files)  # shuffle for variety
    
    mixtape = None
    for i, file in enumerate(files):
        song = AudioSegment.from_file(os.path.join(folder, file))
        song = song.set_channels(2).set_frame_rate(44100)
        
        if mixtape is None:
            mixtape = song
        else:
            # define overlap region
            overlap = min(transition_ms, len(song), len(mixtape))
            
            # outgoing: fade out + high-pass to remove bass
            outgoing = mixtape[-overlap:].fade_out(overlap).low_pass_filter(4000)
            
            # incoming: fade in + low-pass to smooth highs
            incoming = song[:overlap].fade_in(overlap).low_pass_filter(4000)
            
            # overlay transition
            transition = outgoing.overlay(incoming)
            
            mixtape = mixtape[:-overlap] + transition + song[overlap:]
    
    mixtape.export(output, format="mp3")
    print(f"✅ Smooth fade mixtape generated: {output}")

# Example usage
smooth_fade_mixtape("mixtape")


✅ Smooth fade mixtape generated: mixtape2.mp3


BEAT MATCHING

In [3]:
import os
import numpy as np
import librosa
from pydub import AudioSegment
import random

def detect_tempo_and_beats(path):
    """Load audio with librosa and return tempo + beat times (in ms)."""
    y, sr = librosa.load(path, sr=44100, mono=True)
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    tempo = float(tempo)  # ensure scalar
    beat_times = librosa.frames_to_time(beat_frames, sr=sr) * 1000  # ms
    return tempo, beat_times


def get_key(path):
    """Rough key detection via librosa chroma (returns major/minor estimate)."""
    y, sr = librosa.load(path, sr=44100)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_mean = chroma.mean(axis=1)
    pitch_class = chroma_mean.argmax()

    keys_major = ["C", "C#", "D", "D#", "E", "F",
                "F#", "G", "G#", "A", "A#", "B"]
    return keys_major[pitch_class]

def beat_synced_crossfade(outgoing, incoming, beat_times_out, beat_times_in, overlap_beats=4):
    """
    Crossfade outgoing and incoming track aligned to beats.
    - overlap_beats: how many beats to overlap
    """
    # Pick last beat of outgoing, first beat of incoming
    if len(beat_times_out) < overlap_beats or len(beat_times_in) < overlap_beats:
        overlap_ms = 5000  # fallback to 5 sec
    else:
        overlap_ms = int(min(beat_times_out[-1] - beat_times_out[-overlap_beats],
                            beat_times_in[overlap_beats-1] - beat_times_in[0]))
    
    outgoing_seg = outgoing[-overlap_ms:].fade_out(overlap_ms).low_pass_filter(1000)
    incoming_seg = incoming[:overlap_ms].fade_in(overlap_ms).low_pass_filter(3000)
    
    return outgoing_seg.overlay(incoming_seg), overlap_ms

def smart_mixtape(folder, output="mixtape4.mp3"):
    files = [f for f in os.listdir(folder) if f.endswith((".mp3", ".wav"))]
    random.shuffle(files)

    mixtape = None
    for i, file in enumerate(files):
        path = os.path.join(folder, file)
        print(f"Processing: {file}")
        tempo, beats = detect_tempo_and_beats(path)
        key = get_key(path)

        song = AudioSegment.from_file(path).set_channels(2).set_frame_rate(44100)

        print(f"  Tempo: {tempo:.1f} BPM | Key: {key}")


        if mixtape is None:
            mixtape = song
            prev_beats = beats
        else:
            transition, overlap = beat_synced_crossfade(mixtape, song, prev_beats, beats)
            mixtape = mixtape[:-overlap] + transition + song[overlap:]
            prev_beats = beats

    mixtape.export(output, format="mp3")
    print(f"✅ Beat-synced mixtape generated: {output}")

# Example usage
smart_mixtape("mixtape")


Processing: yt1s.com - Awen  Caiiro  Your Voice Adam Port Remix MIDH 043.mp3


C:\Users\djadh\AppData\Local\Temp\ipykernel_3408\2643082961.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tempo = float(tempo)  # ensure scalar


  Tempo: 123.0 BPM | Key: A
Processing: yt1s.com - Waves  Wavs.mp3
  Tempo: 120.2 BPM | Key: A
✅ Beat-synced mixtape generated: mixtape4.mp3


In [ ]:
from pydub import AudioSegment
import os, random

def smooth_fade_mixtape(folder, output="output/mixtape3.mp3", transition_ms=6000):
    files = [f for f in os.listdir(folder) if f.endswith((".mp3", ".wav"))]
    random.shuffle(files)  # shuffle for variety
    
    mixtape = None
    for i, file in enumerate(files):
        song = AudioSegment.from_file(os.path.join(folder, file))
        song = song.set_channels(2).set_frame_rate(44100)
        
        if mixtape is None:
            mixtape = song
        else:
            # define overlap region
            overlap = min(transition_ms, len(song), len(mixtape))
            
            # outgoing: fade out + high-pass to remove bass
            outgoing = mixtape[-overlap:].fade_out(overlap).low_pass_filter(4000)
            
            # incoming: fade in + low-pass to smooth highs
            incoming = song[:overlap].fade_in(overlap).low_pass_filter(4000)
            
            # overlay transition
            transition = outgoing.overlay(incoming)
            
            mixtape = mixtape[:-overlap] + transition + song[overlap:]
    
    mixtape.export(output, format="mp3")
    print(f"✅ Smooth fade mixtape generated: {output}")

# Example usage
smooth_fade_mixtape("mixtape")


✅ Smooth fade mixtape generated: output/mixtape3.mp3


### video integration

In [ ]:
import subprocess
import os

def make_video_from_audio(image_path, audio_path, output_path="output/mixtape_vid3.mp4"):
    """
    Merge a static image with audio to create a video.
    Resizes image to even dimensions (required by libx264).
    Re-encodes audio to AAC for MP4.
    """
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")
    if not os.path.exists(audio_path):
        raise FileNotFoundError(f"Audio not found: {audio_path}")

    cmd = f'ffmpeg -y -loop 1 -i "{image_path}" -i "{audio_path}" ' \
        f'-vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" ' \
        f'-c:v libx264 -preset fast -tune stillimage ' \
        f'-c:a aac -b:a 192k -shortest "{output_path}"'

    try:
        subprocess.run(cmd, shell=True, check=True)
        print(f"✅ Video created: {output_path}")
    except subprocess.CalledProcessError:
        print("❌ FFmpeg failed! Please check image/audio paths and formats.")

# Example usage
image_path = r"images/f.jpg"
audio_path = r"output/mixtape3.mp3"
make_video_from_audio(image_path, audio_path)


Optimized

In [2]:
import subprocess
import os
from PIL import Image

def make_video_from_audio_optimized(image_path, audio_path, output_path="output/mixtape_vid3.mp4",
                                    video_resolution=(1280, 720), fps=1, preset="ultrafast"):
    """
    Merge a static image with audio to create a video.
    Optimized for long audios and high-resolution images.
    - Resizes image to desired resolution (even dimensions for libx264)
    - Uses low fps (1 fps) for static image to reduce processing
    - Uses ultrafast preset for speed
    - Re-encodes audio to AAC for MP4
    """
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")
    if not os.path.exists(audio_path):
        raise FileNotFoundError(f"Audio not found: {audio_path}")

    # Ensure output folder exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Resize image if necessary
    img_resized_path = "temp_resized_image.jpg"
    img = Image.open(image_path)
    img = img.resize(video_resolution)
    img.save(img_resized_path)

    # FFmpeg command
    cmd = [
        "ffmpeg",
        "-y",
        "-loop", "1",
        "-i", img_resized_path,
        "-i", audio_path,
        "-c:v", "libx264",
        "-preset", preset,
        "-tune", "stillimage",
        "-r", str(fps),  # low fps for static image
        "-c:a", "aac",
        "-b:a", "192k",
        "-shortest",
        output_path
    ]

    try:
        subprocess.run(cmd, check=True)
        print(f"✅ Video created: {output_path}")
    except subprocess.CalledProcessError as e:
        print("❌ FFmpeg failed! Error:", e)
    finally:
        # Clean up temporary resized image
        if os.path.exists(img_resized_path):
            os.remove(img_resized_path)

# Example usage
image_path = r"images/hh.jpg"
audio_path = r"output/mixtape3.mp3"
make_video_from_audio_optimized(image_path, audio_path)


✅ Video created: output/mixtape_vid3.mp4


YT description

In [2]:
import os

def generate_youtube_description(track_folder, mixtape_name="Afro House Mixtape", genre="Afro House"):
    """
    Generates a YouTube description for a mixtape based on track filenames.
    """
    # Read all audio files
    audio_extensions = (".mp3", ".wav", ".flac", ".aac", ".ogg", ".m4a")
    tracks = [f for f in os.listdir(track_folder) if f.lower().endswith(audio_extensions)]

    if not tracks:
        return "No audio tracks found in the folder!"

    # Sort tracks (optional: alphabetical)
    tracks.sort()

    # Remove extensions
    tracklist = [os.path.splitext(track)[0] for track in tracks]

    # Generate description
    description = f"🔥 {mixtape_name} 🔥\n"
    description += f"Genre: {genre}\n\n"
    description += "🎵 Tracklist:\n"
    for idx, track in enumerate(tracklist, 1):
        description += f"{idx}. {track}\n"

    description += "\n💽 Download/Listen links:\n"
    description += "You can find these tracks online or on Spotify/Apple Music.\n\n"

    description += "🎧 Follow for more Afro House mixes!\n\n"

    # Hashtags for SEO
    hashtags = [
        "#AfroHouse", "#HouseMusic", "#EDM", "#DanceMusic", "#Mixtape",
        "#ElectronicMusic", "#DJMix", "#PartyMusic", "#DeepHouse", "#AfroHouseVibes"
    ]
    description += " ".join(hashtags)

    return description


# Example usage
track_folder = r"mixtape"  # folder containing mp3s
description_text = generate_youtube_description(track_folder, mixtape_name="Afro House Summer Mix", genre="Afro House")
print(description_text)

# Optional: save to file
with open("youtube_description.txt", "w", encoding="utf-8") as f:
    f.write(description_text)


🔥 Afro House Summer Mix 🔥
Genre: Afro House

🎵 Tracklist:
1. yt1s.com - Awen  Caiiro  Your Voice Adam Port Remix MIDH 043
2. yt1s.com - Burna Boy  Its Plenty KetsoSA Defeat Mix
3. yt1s.com - Chopstar  Glitch Effect Original Mix
4. yt1s.com - Dj Weather10 Days Original Mix
5. yt1s.com - Francis Mercier  Magic System  Premier Gaou Nitefreak Extended Remix
6. yt1s.com - InQfive Dr Feel  White Hyena Original Mix
7. yt1s.com - Keinemusik ME Rampa Adam Port  Feeling feat RY X
8. yt1s.com - Mr Silk Team Distant feat Jinger Stone  Thunderstorm Kususa Remix
9. yt1s.com - SUPTA  Ntinga Ntaka Oficial Audio ft Thalitha  Obie
10. yt1s.com - Safar FR  Feat Dikla Hackmon  Thousand and Two Nights Paso Doble Extended Mix
11. yt1s.com - Sizwe Sakithi (1)
12. yt1s.com - Sizwe Sakithi
13. yt1s.com - Vanco  Cee Elassaad Ft Lizwi  Zamekile Club Edit
14. yt1s.com - Waves  Wavs

💽 Download/Listen links:
You can find these tracks online or on Spotify/Apple Music.

🎧 Follow for more Afro House mixes!

#AfroHous

Yt description with time stamps


In [1]:
import os
from pydub import AudioSegment

def generate_youtube_description_with_timestamps(track_folder, mixtape_name="Afro House Mixtape", genre="Afro House", start_time_sec=0):
    """
    Generates a YouTube description with tracklist, timestamps, and hashtags.
    Assumes audio files are in order of playback.
    """
    audio_extensions = (".mp3", ".wav", ".flac", ".aac", ".ogg", ".m4a")
    tracks = [f for f in os.listdir(track_folder) if f.lower().endswith(audio_extensions)]
    if not tracks:
        return "No audio tracks found in the folder!"

    tracks.sort()  # optional: sort alphabetically

    description = f"🔥 {mixtape_name} 🔥\n"
    description += f"Genre: {genre}\n\n"
    description += "🎵 Tracklist:\n"

    current_time = start_time_sec

    for idx, track in enumerate(tracks, 1):
        track_path = os.path.join(track_folder, track)
        audio = AudioSegment.from_file(track_path)
        duration_sec = len(audio) // 1000  # duration in seconds

        # Format timestamp mm:ss
        minutes = current_time // 60
        seconds = current_time % 60
        timestamp = f"{minutes:02d}:{seconds:02d}"

        track_name = os.path.splitext(track)[0]
        description += f"{timestamp} - {track_name}\n"

        current_time += duration_sec  # increment for next track

    description += "\n💽 Download/Listen links:\n"
    description += "You can find these tracks online or on Spotify/Apple Music.\n\n"
    description += "🎧 Follow for more Afro House mixes!\n\n"

    hashtags = [
        "#AfroHouse", "#HouseMusic", "#EDM", "#DanceMusic", "#Mixtape",
        "#ElectronicMusic", "#DJMix", "#PartyMusic", "#DeepHouse", "#AfroHouseVibes"
    ]
    description += " ".join(hashtags)

    return description

# Example usage
track_folder = r"mixtape"
description_text = generate_youtube_description_with_timestamps(
    track_folder,
    mixtape_name="Afro House Summer Mix",
    genre="Afro House"
)

print(description_text)

# Optional: save to file
with open("youtube_description.txt", "w", encoding="utf-8") as f:
    f.write(description_text)


🔥 Afro House Summer Mix 🔥
Genre: Afro House

🎵 Tracklist:
00:00 - yt1s.com - Awen  Caiiro  Your Voice Adam Port Remix MIDH 043
04:20 - yt1s.com - Burna Boy  Its Plenty KetsoSA Defeat Mix
10:30 - yt1s.com - Chopstar  Glitch Effect Original Mix
16:10 - yt1s.com - Dj Weather10 Days Original Mix
22:59 - yt1s.com - Francis Mercier  Magic System  Premier Gaou Nitefreak Extended Remix
29:23 - yt1s.com - InQfive Dr Feel  White Hyena Original Mix
36:14 - yt1s.com - Keinemusik ME Rampa Adam Port  Feeling feat RY X
41:18 - yt1s.com - Mr Silk Team Distant feat Jinger Stone  Thunderstorm Kususa Remix
48:08 - yt1s.com - SUPTA  Ntinga Ntaka Oficial Audio ft Thalitha  Obie
53:50 - yt1s.com - Safar FR  Feat Dikla Hackmon  Thousand and Two Nights Paso Doble Extended Mix
61:38 - yt1s.com - Sizwe Sakithi (1)
69:18 - yt1s.com - Sizwe Sakithi
76:58 - yt1s.com - Vanco  Cee Elassaad Ft Lizwi  Zamekile Club Edit
85:22 - yt1s.com - Waves  Wavs

💽 Download/Listen links:
You can find these tracks online or on Spo